In [5]:
class Requester(object):   
# base class  
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.84 Safari/537.36',

    }
    
    def req(self,url):
        connection = True
        import requests
        from bs4 import BeautifulSoup
        import time
        soup=''
        rs = requests.session()
        while connection == True:           
            try:                
                res = rs.get(url,headers=self.headers)                
                soup = BeautifulSoup(res.text)                
                connection = False
                
                return soup
            except:
                print ('connection error , sleep 1 min' )                         
                time.sleep(60)
class Crawler(object):
    currentCrawlingTag = ''
    domain = 'http://www.wakema.com.tw/'
    rs = Requester()    
    def tagCrawl(self,url,wantedTag):
        soup = self.rs.req(url)
        linkList = []
        for link in soup.select(wantedTag):
            linkList.append(self.domain+link['href'])
        return linkList
    def storeUrlCrawl(self,url):
        import re
        result = []
        currentTag = ''
        hasNext = True         
        # example url = http://www.wakema.com.tw/category/CategoryBottom.jsp?t158_id=68&&pageNo=1
        while hasNext == True:
            try:
                soup = self.rs.req(url)
                if currentTag == '':
                    currentTag=soup.select('.breadcrumb li')[-2].a.text+'@%&'+soup.select('.breadcrumb li')[-1].a.text
                stores=soup.select('.col-lg-10.col-md-9.col-xs-12 h4 a')

                for store in stores :
                    result.append(self.domain+store['href'])
                url = self.domain+soup.find_all(alt=re.compile('下一頁'))[0].parent['href']                
            except:
                print('no next page ,{} done .'.format(currentTag))
                self.currentCrawlingTag = currentTag.strip('\ufeff')
                hasNext = False
        return result
    def contentCrawl(self,urlList):
        import re
        result = []        
        for url in urlList:
            soup = self.rs.req(url)
            #電話
            try:
                tel=soup.find_all(href=re.compile("tel:"))[0]['href'].split(':')[1]
            except IndexError:
                print(url,'cant find tel .')
            #所有我能找的到的tag    note: tag應該和其他資訊使用不同的分隔，這個網站tag數量會變化
            tagObjects = soup.select('.table tr')[3].select('a')
            tag = ''
            for tagObject in tagObjects:                
                tag +=tagObject.text+'@%&'
            tag = tag+self.currentCrawlingTag            
            #地址
            address = soup.find_all(itemprop=re.compile('address'))[0].text
            #店名
            store_name = soup.find_all(itemprop=re.compile('name'))[0].text
            #官網 沒有官網的紀錄為None
            try:
                officialSite = soup.select('.text-primary a')[0]
                officialSite_Url = officialSite['href']                
            except IndexError :
                officialSite_Url = 'None'
            #商店介紹
            store_intro = soup.select('.alicef2')[0].text
            #服務項目
            service_intro = soup.select('.panel-body')[1].p.text
            
            result.append(tag+'_|'+store_name+'_|'+tel+'_|'+address+'_|'+store_intro+'_|'+service_intro+'_|'+officialSite_Url+'\n')
            
        return result
def exitfunc():
    print('program finish , clean the doneList. ')
    import os
    os.remove('doneList.txt')          

In [ ]:
import sys
urlList = []
linkList = []
tc = Crawler()
try:
    g = open('wakemaDoneList.txt','r' , encoding='utf-8')
except:
    with open('wakemaDoneList.txt','a+' , encoding='utf-8') as justCreate:
        justCreate.close()
    g = open('wakemaDoneList.txt','r' , encoding='utf-8')
# 註冊回調函數，如果這隻程式做成.py檔，應該會在執行完畢的時候刪除doneList... 應該啦。
sys.exitfunc=exitfunc
listBeforeStrip=g.read().splitlines()
doneList=list(map(str.strip, listBeforeStrip))
g.close()
topicList = tc.tagCrawl('http://www.wakema.com.tw/','.gold_mu a')
for categoryLink  in topicList :    
    categoryList = tc.tagCrawl(categoryLink,'.col-md-6.col-xs-12 a')
    for actegoryLink in categoryList:        
        urlList=tc.storeUrlCrawl(actegoryLink)
        print(doneList , tc.currentCrawlingTag)
        if tc.currentCrawlingTag in doneList:
            print(tc.currentCrawlingTag,'crawled berfore , skip it ')
        result = tc.contentCrawl(urlList)
        with open('wakema.txt','a' ,encoding='utf-8') as wakema:
            for line in result:
                wakema.write(line)
        wakemaDoneList = open('wakemaDoneList.txt','a',encoding='utf-8')
        wakemaDoneList.write(tc.currentCrawlingTag+'\n')
        wakemaDoneList.close()

        

C:\Users\michaeltsai\AppData\Local\Continuum\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


no next page ,餐飲設備裝置@%&餐飲設備 done .
['\ufeff餐飲設備裝置@%&餐飲設備', '餐飲設備裝置@%&電解水機', '餐飲設備裝置@%&製冰機', '餐飲設備裝置@%&淨水器'] 餐飲設備裝置@%&餐飲設備
no next page ,餐飲設備裝置@%&電解水機 done .
['\ufeff餐飲設備裝置@%&餐飲設備', '餐飲設備裝置@%&電解水機', '餐飲設備裝置@%&製冰機', '餐飲設備裝置@%&淨水器'] 餐飲設備裝置@%&電解水機
餐飲設備裝置@%&電解水機 crawled berfore , skip it 
no next page ,餐飲設備裝置@%&製冰機 done .
['\ufeff餐飲設備裝置@%&餐飲設備', '餐飲設備裝置@%&電解水機', '餐飲設備裝置@%&製冰機', '餐飲設備裝置@%&淨水器'] 餐飲設備裝置@%&製冰機
餐飲設備裝置@%&製冰機 crawled berfore , skip it 
connection error , sleep 1 min


In [1]:
print('\ufeff')

﻿
